In [1]:
# Core
import numpy as np
import pandas as pd

# Tensorflow / ML
import tensorflow as tf

# Utility
import os

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl

# Utility
import time

In [2]:
# Set visibible GPU
gpu_num: int = 1
os.environ['CUDA_VISIBLE_DEVICES']=f'{gpu_num}'

In [3]:
# Configure TF GPU growth
import kepler_sieve
from tf_utils import gpu_grow_memory, get_gpu_device
gpu_grow_memory(verbose=True)

Found 0 GPUs.  Setting memory growth = True.


In [4]:
# MSE Imports
from asteroid_search import load_fitted_elts, load_ztf_hits, append_fitted_elt, append_ztf_hit, file_name_model
from asteroid_element import load_ast_elt
from candidate_element import asteroid_elts, perturb_elts, random_elts, elts_add_mixture_params, elts_add_H
from random_elements import load_best_random_elts, make_ztf_ast
from ztf_ast import load_ztf_nearest_ast, calc_hit_freq
from ztf_element import load_ztf_batch, make_ztf_batch, ztf_score_by_elt, ztf_elt_summary
from asteroid_model import AsteroidPosition, AsteroidDirection, make_model_ast_pos
from asteroid_search_layers import CandidateElements, MixtureParameters, TrajectoryScore
from asteroid_search_model import AsteroidSearchModel
from asteroid_search_report import traj_diff
from nearest_asteroid import nearest_ast_elt_cart, nearest_ast_elt_cov, elt_q_norm
from element_eda import score_by_elt
from asteroid_dataframe import calc_ast_data, spline_ast_vec_df
from astro_utils import deg2dist, dist2deg, dist2sec

In [5]:
# Aliases
keras = tf.keras

# Constants
dtype = tf.float32
dtype_np = np.float32
space_dims = 3

In [6]:
# Set plot style variables
mpl.rcParams['figure.figsize'] = [16.0, 10.0]
mpl.rcParams['font.size'] = 16

In [7]:
# Load orbital elements for known asteroids
ast_elt = load_ast_elt()

# Number of asteroids
N_ast = ast_elt.shape[0]

## Review Fitted Elements

In [8]:
known_ast = True

In [9]:
fitted_elts = load_fitted_elts(known_ast=known_ast, min_hits=8)
best_element_id = fitted_elts.element_id.iloc[0]

In [10]:
fitted_elts

,element_id,a,e,inc,Omega,omega,f,epoch,num_hits,R_sec,thresh_sec,log_like,hits,num_rows_close,timestamp
element_id,,,,,,,,,,,,,,,
13172,13172,2.574177,0.177445,0.172849,2.846966,2.930934,-5.419372,58600.0,72.941956,6.534205,445.092957,474.436218,65.986427,95.0,2020-04-24 08:08:48.008207
57218,57218,2.896754,0.044306,0.014004,4.265533,0.151008,2.625664,58600.0,84.132111,12.023473,625.552429,391.802307,58.903355,263.0,2020-04-25 21:50:46.937329
52563,52563,2.687424,0.131536,0.235721,6.100040,3.748723,2.988849,58600.0,130.984573,9.423238,486.837982,812.298706,50.996090,141.0,2020-04-26 10:09:24.512576
4754,4754,2.313755,0.036114,0.132387,1.235678,1.532763,-1.881582,58600.0,91.929352,14.014965,934.127380,514.902466,40.964291,247.0,2020-04-24 01:53:29.452031
67488,67488,3.064386,0.204280,0.120861,0.672962,3.007123,-2.987987,58600.0,113.605659,67.432640,1897.126709,412.509308,32.399624,943.0,2020-04-26 05:40:37.113303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52895,52895,3.132184,0.119650,0.164686,5.829597,5.651721,2.826113,58600.0,59.932625,58.877975,1325.285645,138.092133,7.760432,524.0,2020-04-25 18:27:58.838723
63886,63886,2.894544,0.030639,0.042188,4.768021,2.224784,-0.188587,58600.0,62.725899,106.883820,2186.848633,70.682877,7.353265,1215.0,2020-04-26 02:56:52.339982
67200,67200,2.608313,0.101753,0.019899,1.924526,3.819631,1.049037,58600.0,79.663246,130.074417,2260.503906,139.011566,7.187671,1439.0,2020-04-26 05:27:25.352868


In [11]:
fitted_elts.shape[0]

102

In [12]:
fitted_elts.loc[best_element_id]

element_id                             13172
a                                    2.57418
e                                   0.177445
inc                                 0.172849
Omega                                2.84697
omega                                2.93093
f                                   -5.41937
epoch                                  58600
num_hits                              72.942
R_sec                                6.53421
thresh_sec                           445.093
log_like                             474.436
hits                                 65.9864
num_rows_close                            95
timestamp         2020-04-24 08:08:48.008207
Name: 13172, dtype: object

In [ ]:
np.mean(fitted_elts.R_sec)

In [ ]:
# ast_elt.loc[best_element_id]

In [ ]:
# fitted_elts.to_csv('../data/candidate_elt/fitted_elts_unknown.csv')

## Review ZTF Hits

In [ ]:
ztf_hits = load_ztf_hits(known_ast=known_ast, display=True, min_hits=3)

In [ ]:
ztf_hits.loc[best_element_id]

In [ ]:
cols = ['mjd', 'ra', 'dec', 'mag_app', 's_sec']
ztf_hits[cols].loc[best_element_id]

In [ ]:
np.mean(ztf_hits.s_sec.loc[best_element_id])

In [ ]:
ztf_id_close = ztf_hits.ztf_id.loc[best_element_id]

In [ ]:
# ztf_hits.to_csv('../data/candidate_elt/ztf_hits_known.csv')

## Load ZTF Data and Batch of Orbital Elements

In [ ]:
# Parameters to build random orbital elements and ztf_ast
batch_size_init = 1024
batch_size = 64
random_seed = best_element_id // batch_size
thresh_deg = 2.0

In [ ]:
# Load ztf nearest asteroid data
ztf_ast = make_ztf_ast(known_ast=known_ast)

In [ ]:
# # Review ztf_ast
# ztf_ast

In [ ]:
# Best random elements
elts = load_best_random_elts(random_seed=random_seed, known_ast=known_ast, 
                             batch_size_init=batch_size_init, batch_size=batch_size, thresh_deg=thresh_deg)

In [ ]:
# # Review best random elements
# elts

In [ ]:
ztf_ast.loc[ztf_id_close]

In [ ]:
close_ast_num = ztf_ast.nearest_ast_num.loc[ztf_id_close].mode().values[0]
close_ast_num

In [ ]:
nearest_ast_dist = ztf_ast.nearest_ast_dist.loc[ztf_id_close]

In [ ]:
dist2sec(nearest_ast_dist)

## Batches of ZTF Data Near Initial Candidate Elements

In [ ]:
# Arguments to make_ztf_batch
near_ast = False
regenerate = False

In [ ]:
# Load perturbed element batch
ztf_elt = load_ztf_batch(elts=elts, ztf=ztf_ast, thresh_deg=thresh_deg, near_ast=near_ast, regenerate=regenerate)

In [ ]:
# # Review ZTF elements
# ztf_elt

In [ ]:
# Mixture parameters
num_hits: int = 10
R_deg: float = 0.5

In [ ]:
# Add mixture parameters to candidate elements
elts_add_mixture_params(elts=elts, num_hits=num_hits, R_deg=R_deg, thresh_deg=thresh_deg)

In [ ]:
# Add brightness parameter H
elts_add_H(elts=elts)

In [ ]:
# # Review perturbed elements; includes nearest asteroid number and distance
# elts

## Build Asteroid Search Model

In [ ]:
# Observatory for ZTF data is Palomar Mountain
site_name = 'palomar'

In [ ]:
# Training parameters
learning_rate = 2.0**-12
clipnorm = 1.0

In [ ]:
# The file name for the saved model
file_name = file_name_model(seed=random_seed, known_ast=known_ast, 
                            batch_size_init=batch_size_init, batch_size=batch_size, thresh_deg=thresh_deg)

In [ ]:
# Build asteroid search model
model = AsteroidSearchModel(
                elts=elts, ztf_elt=ztf_elt, 
                site_name=site_name, thresh_deg=thresh_deg, 
                learning_rate=learning_rate, clipnorm=clipnorm,
                name='model',
                file_name=file_name,
)

In [ ]:
# Load trained model
model.load()

In [ ]:
# Report before training starts
model.report()

In [ ]:
# Generate fitted elements
elts_fit = model.candidates_df()

In [ ]:
cols = ['element_id', 'a', 'e', 'inc', 'Omega', 'omega', 'f', 'epoch',
        'num_hits', 'R', 'R_deg', 'R_sec', 'R_max', 'R_deg_max', 'thresh_s',
        'thresh_deg', 'thresh_sec', 'log_like', 'hits', 'num_rows_close']

In [ ]:
elts_fit[cols]

In [ ]:
# Save ztf_hits to ztf_hits_known_ast or ztf_unknown_ast

In [ ]:
# Generate ZTF hits
ztf_hit = model.calc_ztf_hits()

In [ ]:
# ztf_hit

In [ ]:
elts_fit, elts_near_ast = model.nearest_ast()

In [ ]:
# Review asteroids nearest to the fitted elements
cols = ['element_id', 'log_like', 'hits', 'R_sec', 'thresh_sec', 'nearest_ast_num', 'nearest_ast_dist', 'nearest_ast_q_norm']

# Summary
has_hits = (model.elts_fit.hits >= 5)
is_converged = has_hits
num_converged = np.sum(is_converged)

# Mean on converged
mean_hits = np.mean(model.elts_fit.hits[is_converged])
mean_R_sec = np.mean(model.elts_fit.R_sec[is_converged])
err_cart_mean = np.mean(model.elts_near_ast.nearest_ast_dist[is_converged])
err_cart_geo = np.exp(np.mean(np.log(model.elts_near_ast.nearest_ast_dist[is_converged])))
err_cov_mean = np.exp(np.mean(np.log(model.elts_near_ast.nearest_ast_q_norm[is_converged])))

# Report
print(f'Good convergence on {num_converged} asteroids.')
print(f'Mean on converged asteroids:')
print(f'Number of hits         : {mean_hits:5.2f}')
print(f'Resolution in arc sec  : {mean_R_sec:5.2f}')
print(f'GeoMean Distance in AU : {err_cart_geo:5.2e}')
print(f'Covariance norm        : {err_cov_mean:5.2e}')

In [ ]:
# Review good matches
model.elts_fit[cols][is_converged].sort_values(by=['hits', 'R_sec'], ascending=[False, True])